In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit 
from sklearn.preprocessing import OneHotEncoder

In [2]:
# 参数设置
LEARNING_RATE = 1e-5
INPUT_NODE = 24
OUTPUT_NODE = 10  # 这里注意修改=============================

LAYER_NODE = [200, 100]
BATCH_SIZE = 50
MAX_EPOCH = 1000000
MODE_NUM = MAX_EPOCH / 100

In [3]:
#define converts(字典)  
def Name_label(s):
#     it = {b'cjn':0, b'lmx':1, b'yxd':2, b'zzy':3 }
#     return it[s]
    return int(s) - 1

In [4]:
csv_path = '../gen_data/dynamic_features.csv'
column_name = ['A_pos_peak_time', 'MAX_pos_peak_time', 'MIN_pos_peak_time', 'A_neg_peak_time', 'MAX_neg_peak_time', 'MIN_neg_peak_time',
 'A_close time', 'MAX_close time', 'MIN_close time', 'A_open time', 'MAX_open time', 'MIN_open time', 'A_blink time', 
 'MAX_blink time', 'MIN_blink time', 'A_peak ratio', 'MAX_peak ratio', 'MIN_peak ratio', 'A_peak dist ratio', 'MAX_peak dist ratio', 
 'MIN_peak dist ratio', 'A_area ratio', 'MAX_area ratio', 'MIN_area ratio', 'Label']
data = pd.read_csv(csv_path)
data.head()
from sklearn.model_selection import StratifiedShuffleSplit 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=25)
for train_index, test_index in split.split(data, data["Label"]): 
    strat_train_set = data.loc[train_index] 
    strat_test_set = data.loc[test_index]
strat_train_set = np.array(strat_train_set)
strat_test_set = np.array(strat_test_set)
train_data,train_label = np.split(strat_train_set,indices_or_sections=(24,),axis=1) #x为数据，y为标签  前24个为特征，最后一个为标签
test_data, test_label = np.split(strat_test_set,indices_or_sections=(24,),axis=1) #x为数据，y为标签  前24个为特征，最后一个为标签

# one-hot covert

In [5]:
encoder = OneHotEncoder()
# print(train_label.reshape(-1,))
train_label_ = encoder.fit_transform(train_label.reshape(-1,1))
train_label = np.array(train_label_.toarray())

test_label_ = encoder.fit_transform(test_label.reshape(-1,1))
test_label = np.array(test_label_.toarray())

d:\tfp36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
d:\tfp36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this On

In [6]:
# print(np.shape(a))
# print(np.shape(b))
# print(np.shape(train_label_2.toarray()))
# print(np.shape(test_label_2.toarray()))
# train_label_2.toarray()


# 训练模型

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.01, shape=shape)
    return tf.Variable(initial)


def createNetwork():
#     定义神经网络的参数和偏置
    w_fc1 = weight_variable([INPUT_NODE, LAYER_NODE[0]])
    b_fc1 = bias_variable([LAYER_NODE[0]])
    
    w_fc2 = weight_variable([LAYER_NODE[0], LAYER_NODE[1]])
    b_fc2 = bias_variable([LAYER_NODE[1]])
    
#     w_fc3 = weight_variable([LAYER_NODE[1], LAYER_NODE[2]])
#     b_fc3 = bias_variable([LAYER_NODE[2]])
    
    w_fc3 = weight_variable([LAYER_NODE[1], OUTPUT_NODE])
    b_fc3 = bias_variable([OUTPUT_NODE])
    
#     输入层
    s = tf.placeholder("float", [None, INPUT_NODE])
    
#     隐藏层
    fc1 = tf.nn.relu(tf.matmul(s, w_fc1) + b_fc1)
    fc2 = tf.nn.relu(tf.matmul(fc1, w_fc2) + b_fc2)
#     fc3 = tf.nn.relu(tf.matmul(fc2, w_fc3) + b_fc3)
    readout = tf.nn.softmax(tf.matmul(fc2, w_fc3) + b_fc3)
    
    tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_fc1)
    tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_fc2)
    tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_fc3)

    
    print("end of create network")
    return s, readout

In [8]:
def trainNetwork(s, readout, sess, max_epoch=MAX_EPOCH, lr=LEARNING_RATE):
    y_ = tf.placeholder("float", [None, OUTPUT_NODE])
    x = s
    y = readout
    
    train_data_length = len(train_data)
    print("train_data_length: ", train_data_length)
    
#     cross_entropy = -tf.reduce_mean(y_*tf.log(y))
#     避免因为出现 0*log(0) 导致的Nan
    cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1.0)))
    regularizer = tf.contrib.layers.l2_regularizer(scale=5.0 / train_data_length)
    reg_term = tf.contrib.layers.apply_regularization(regularizer)
    cross_entropy_ = cross_entropy + reg_term
    
    
    y_argmax = tf.argmax(input=y,axis=1)
    y_argmax_ = tf.argmax(input=y_,axis=1)
    
    
    eq_ = tf.equal(y_argmax, y_argmax_)
    eq_ = tf.cast(eq_, dtype=tf.float32) # 这里不能为int
    acc = tf.reduce_mean(eq_)
    
    
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy_)
    
    # 用于加载或保存网络参数
    saver = tf.train.Saver(max_to_keep=3)
    sess.run(tf.initialize_all_variables())
    checkpoint = tf.train.get_checkpoint_state("saved_networks")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")
        
    
#     data_path = '../gen_data/dynamic_feature-2-26.data'
#     data = np.loadtxt(data_path, dtype=float, delimiter=',', converters={24:Name_label})
#     # 划分数据与标签
#     total_data, total_label = np.split(data,indices_or_sections=(24,),axis=1) # 前24个为特征，最后一个为标签

    # 将标签转化成 one-hot形式
    # 行向量转列向量
#     print(total_label)
#     total_label = np.array(total_label).reshape(len(total_label), -1)
#     enc = OneHotEncoder()
#     enc.fit(total_label)
#     total_label = enc.transform(total_label).toarray()
# #     print("============")
# #     print(total_label)
#     print("one-hot shape:", np.shape(total_label))
    
    
#     # 指定随机数种子，确保每次划分结果一致
#     train_data,test_data,train_label,test_label = train_test_split(total_data, total_label, random_state=42, 
#                                                                    train_size=0.8, test_size=0.2) #sklearn.model_selection.
    
    training_epoch = 0

    
    acc_list =[]
    val_list =[]
    while training_epoch <= max_epoch:
        # 数据的选取
        valid_indices = np.random.choice(train_data_length, [BATCH_SIZE], replace=False)
        batch_train_data = train_data[valid_indices]
        batch_train_label = train_label[valid_indices]
        
#         debug ==========
#         print("batch_train_data", np.shape(batch_train_data))
#         print("batch_train_label", np.shape(batch_train_label))
        
        loss, _ = sess.run([cross_entropy_, train_step], feed_dict={s: batch_train_data, y_: batch_train_label})
        
        if training_epoch % MODE_NUM == 0:
            # validation set
#             valid_set_indices = np.random.choice(train_data_length, [BATCH_SIZE], replace=False)
#             valid_set_data = train_data[valid_set_indices]
#             valid_set_label = train_label[valid_set_indices]
            
            acc_ = sess.run([acc,], feed_dict={s: test_data, y_: test_label})
            acc_ = acc_[0]
            
            val_ = sess.run([acc,], feed_dict={s: train_data, y_: train_label})
            val_ = val_[0]
            
            print("epoch=%d, loss=%.5f, valid_acc=%.5f, test_acc=%.5f" % (training_epoch, loss, val_, acc_))
            acc_list.append(acc_)
            val_list.append(val_)
        
#         每进行500次迭代，保留一下网络参数
        if (training_epoch + 1) % 100000 == 0:
            saver.save(sess, 'saved_networks/od', global_step=training_epoch)
        
        training_epoch += 1
    
    print("This time training has finished! Total training epoch are %d" % (training_epoch - 1, ))
    
    return acc_list, val_list

In [9]:
def predictNetwork():
    a = 2

In [10]:
def analyst_plot(lst, delta=101):
#     训练过程中accuracy的变化过程
    epoch = np.linspace(0, MAX_EPOCH, delta)
    acc, val = np.array(lst)
    
    plt.title("Learning Curve")
    # plt.legend(loc="upper right", fontsize=14)  # not shown in the book
    plt.xlabel("Training set size", fontsize=14)  # not shown
    plt.ylabel("accuracy", fontsize=14)  # not shown

    plt.plot(epoch, acc, color='red', label='train')
    plt.plot(epoch, val, color='blue', label='val')
    plt.legend(loc="best", fontsize=14)   # not shown in the book

#     plt.show()
    plt.savefig("../myplot/mlp_learning_curve.png")  # 图像保存
    plt.close()

In [11]:
if __name__ == "__main__":
    s, readout = createNetwork()
    sess = tf.InteractiveSession()
    mlst = trainNetwork(s, readout, sess)
    analyst_plot(mlst)

end of create network
train_data_length:  210
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Could not find old network weights
epoch=0, loss=0.25293, valid_acc=0.09524, test_acc=0.07547
epoch=10000, loss=0.17945, valid_acc=0.49048, test_acc=0.52830
epoch=20000, loss=0.18255, valid_acc=0.53810, test_acc=0.60377
epoch=30000, loss=0.16043, valid_acc=0.54286, test_acc=0.62264
epoch=40000, loss=0.17296, valid_acc=0.58095, test_acc=0.62264
epoch=50000, loss=0.15521, valid_acc=0.57619, test_acc=0.62264
epoch=60000, loss=0.16042, valid_acc=0.59048, test_acc=0.62264
epoch=70000, loss=0.16292, valid_acc=0.59524, test_acc=0.62264
epoch=80000, loss=0.14873, valid_acc=0.61429, test_acc=0.64151
epoch=90000, loss=0.16296, valid_acc=0.62381, test_acc=0.64151
epoch=100000, loss=0.16226, valid_acc=0.62381, test_acc=0.66038
epoch=110000, loss=0.15071, valid_acc=0.62857, test_acc=0.66038
epoch=120000, loss=0.16219, valid_acc=0.63333, test_acc=0.64151
epoch=130000, loss=0.15564,